In [1]:
import nltk
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [171]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from nltk import ngrams
import re

In [67]:
from nltk.corpus import stopwords
stop_words=set(stopwords.words("english"))

In [88]:
from nltk.stem.wordnet import WordNetLemmatizer
lem = WordNetLemmatizer()

In [107]:
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')

In [114]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_trans = TfidfTransformer(smooth_idf=True, use_idf=True)

In [115]:
from sklearn.feature_extraction.text import CountVectorizer
cvec = CountVectorizer(max_df=.85, stop_words=stopwords)

In [3]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
SIA = SentimentIntensityAnalyzer()

## Load & inspect datasets

Dataset

In [18]:
df_clean_pmc = pd.read_csv("data/clean_pmc.csv", skiprows=0, header=None)
df_clean_noncomm_use = pd.read_csv("data/clean_noncomm_use.csv", skiprows=1, header=None)
df_clean_comm_use = pd.read_csv("data/clean_comm_use.csv", skiprows=1, header=None)
df_biorxiv_clean = pd.read_csv("data/biorxiv_clean.csv", skiprows=1, header=None)

In [19]:
df_concat = pd.concat([df_clean_pmc, df_clean_comm_use, df_clean_noncomm_use, df_biorxiv_clean])

In [20]:
data = df_concat.to_numpy()

In [21]:
data.shape

(13203, 9)

In [22]:
data[0]

array(['paper_id', 'title', 'authors', 'affiliations', 'abstract', 'text',
       'bibliography', 'raw_authors', 'raw_bibliography'], dtype=object)

In [23]:
df_drug_ratings = pd.read_csv("data/drugsComTest_raw.csv", skiprows=0, header=None)

In [24]:
drug_ratings = df_drug_ratings.to_numpy()

In [25]:
drug_ratings.shape

(53767, 7)

In [26]:
drug_ratings[1]

array(['163740', 'Mirtazapine', 'Depression',
       '"I&#039;ve tried a few antidepressants over the years (citalopram, fluoxetine, amitriptyline), but none of those helped with my depression, insomnia &amp; anxiety. My doctor suggested and changed me onto 45mg mirtazapine and this medicine has saved my life. Thankfully I have had no side effects especially the most common - weight gain, I&#039;ve actually lost alot of weight. I still have suicidal thoughts but mirtazapine has saved me."',
       '10', '28-Feb-12', '22'], dtype=object)

In [27]:
drugs = np.hstack([np.unique(drug_ratings[1:,1]), np.array(['chloroquine'])])
conditions = np.unique(drug_ratings[1:,2].astype(str))[44:]

In [28]:
print(drugs.shape, conditions.shape)

(2638,) (665,)


In [147]:
def get_snippet(index, doc):
    return doc[max(index.start()-100, 0):max(index.start()-1, 0)] +\
            doc[index.end():min(index.end()+100, len(doc))]

In [186]:
def process(drug, doc):
    s = 0
    d = ""
    for index in re.finditer(drug.lower(), str(doc)):
        snippet = get_snippet(index, doc)
        sentiment = SIA.polarity_scores(snippet)
        s = sentiment['compound']
        d += ' '.join([' '.join(w) for w in nltk.everygrams([lem.lemmatize(w) for w in tokenizer.tokenize(snippet) if not w in stop_words], max_len=3)])
        
    return s, d

In [187]:
sentimemts = []
documents = []

print("Total:", len(drugs))
for i, drug in enumerate(drugs):
    print("Progress:", i, end="\r")
    s = []
    d = ""
    for doc in data[1:]:
        p = process(drug, doc[1])
        if len(p[1]) > 0: 
            s.append(p[0])
            d += p[1]
        p = process(drug, doc[4])
        if len(p[1]) > 0: 
            s.append(p[0])
            d += p[1]
        p = process(drug, doc[5])
        if len(p[1]) > 0: 
            s.append(p[0])
            d += p[1]
    
    sentimemts.append(s)
    documents.append(d)  

Total: 2638


KeyboardInterrupt: 

In [182]:
documents[256]

['LGP2',
 'construct',
 'deficient',
 'RNA',
 'binding',
 'individual',
 'domain',
 'alone',
 'capable',
 'ofg',
 'MDA5',
 'directed',
 'signaling',
 'Pippig',
 'et',
 'al',
 '2009',
 'Also',
 'supporting',
 'possible',
 'unique',
 'link',
 'MD',
 'LGP2 construct',
 'construct deficient',
 'deficient RNA',
 'RNA binding',
 'binding individual',
 'individual domain',
 'domain alone',
 'alone capable',
 'capable ofg',
 'ofg MDA5',
 'MDA5 directed',
 'directed signaling',
 'signaling Pippig',
 'Pippig et',
 'et al',
 'al 2009',
 '2009 Also',
 'Also supporting',
 'supporting possible',
 'possible unique',
 'unique link',
 'link MD',
 'LGP2 construct deficient',
 'construct deficient RNA',
 'deficient RNA binding',
 'RNA binding individual',
 'binding individual domain',
 'individual domain alone',
 'domain alone capable',
 'alone capable ofg',
 'capable ofg MDA5',
 'ofg MDA5 directed',
 'MDA5 directed signaling',
 'directed signaling Pippig',
 'signaling Pippig et',
 'Pippig et al',
 'et a

In [183]:
wordcounts = cvec.fit_transform(documents)

TypeError: 'WordListCorpusReader' object is not iterable

In [157]:
print(documents[256])

[[['LGP2', 'construct', 'deficient', 'RNA', 'binding', 'individual', 'domain', 'alone', 'capable', 'ofg', 'MDA5', 'directed', 'signaling', 'Pippig', 'et', 'al', '2009', 'Also', 'supporting', 'possible', 'unique', 'link', 'MD'], [('LGP2', 'construct'), ('construct', 'deficient'), ('deficient', 'RNA'), ('RNA', 'binding'), ('binding', 'individual'), ('individual', 'domain'), ('domain', 'alone'), ('alone', 'capable'), ('capable', 'ofg'), ('ofg', 'MDA5'), ('MDA5', 'directed'), ('directed', 'signaling'), ('signaling', 'Pippig'), ('Pippig', 'et'), ('et', 'al'), ('al', '2009'), ('2009', 'Also'), ('Also', 'supporting'), ('supporting', 'possible'), ('possible', 'unique'), ('unique', 'link'), ('link', 'MD')], [('LGP2', 'construct', 'deficient'), ('construct', 'deficient', 'RNA'), ('deficient', 'RNA', 'binding'), ('RNA', 'binding', 'individual'), ('binding', 'individual', 'domain'), ('individual', 'domain', 'alone'), ('domain', 'alone', 'capable'), ('alone', 'capable', 'ofg'), ('capable', 'ofg', '

In [17]:
wordcounts = []
for i, d in enumerate(documents):
    wordcounts.append(Counter(d))

In [18]:
outs = []
for drug in drugs:
    num = 0
    for i, w in enumerate(wordcounts):
        num += sentiments[i]w[drug.lower()]
    outs.append((num, drug))

In [60]:
sorted(outs, reverse=True)

[(3348, 'Glucose'),
 (2035, 'Ribavirin'),
 (1755, 'Oseltamivir'),
 (1186, 'Urea'),
 (1123, 'chloroquine'),
 (1122, 'Lysine'),
 (908, 'Silver'),
 (631, 'Adenosine'),
 (588, 'Luminal'),
 (587, 'Tetracycline'),
 (577, 'Zanamivir'),
 (558, 'Copper'),
 (540, 'Amantadine'),
 (450, 'Tryptophan'),
 (441, 'Doxycycline'),
 (433, 'Biotin'),
 (420, 'Gentamicin'),
 (388, 'Azithromycin'),
 (370, 'Phenol'),
 (350, 'Dexamethasone'),
 (327, 'Fluoride'),
 (321, 'Doxorubicin'),
 (300, 'Aspirin'),
 (288, 'Prednisolone'),
 (282, 'Vancomycin'),
 (265, 'Ketamine'),
 (257, 'Amoxicillin'),
 (253, 'Acyclovir'),
 (250, 'Methylprednisolone'),
 (208, 'Propofol'),
 (206, 'Cyclosporine'),
 (204, 'Cetuximab'),
 (202, 'Ciprofloxacin'),
 (196, 'Imiquimod'),
 (185, 'Tamoxifen'),
 (185, 'Acidophilus'),
 (184, 'Imatinib'),
 (182, 'Metronidazole'),
 (173, 'Prednisone'),
 (171, 'Rosiglitazone'),
 (171, 'Midazolam'),
 (171, 'Hydrocortisone'),
 (170, 'Montelukast'),
 (168, 'Erythromycin'),
 (158, 'Caffeine'),
 (157, 'Echinace

In [105]:
SIA.polarity_scores("just some good old boring talk")['compound']

0.1531

In [189]:
' '.join(['1','2','3'])

'1 2 3'